In [ ]:
import torch
from datasets import Dataset
tokenized_data = torch.load("../data/processed/tokenized_data.pt")
labels = torch.load("../data/processed/labels.pt")
# Convert to Dataset format
dataset = Dataset.from_dict({
    'input_ids': tokenized_data['input_ids'],
    'attention_mask': tokenized_data['attention_mask'],
    'labels': labels
})

# Split dataset into train and test
datasets = dataset.train_test_split(test_size=0.1)
train_dataset = datasets['train']
eval_dataset = datasets['test']
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load model and set training arguments
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)
training_args = TrainingArguments(
    output_dir='../models',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()
trainer.save_model("../models/final_model")
